In [2]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-c.cari.com.my-noisy-translation/resolve/main/filtered.jsonl -O c.cari.com.my.jsonl

In [3]:
!head -n 3 c.cari.com.my.jsonl

["\u56de\u590d 1# jeffreylee  \u53ef\u4ee5\u7528\u4e00\u9910 value meal \u4f86\u63db\u55ce\uff1f", {"english": "Can I exchange it with a value meal?", "malay": "Bolehkah saya menukar ini dengan hidangan nilai?"}]
[" xuesha\u7684\u8138\u597d\u957f\u554a\u3002\u3002\u3002  10507975_338882742936189_1743878600_n.jpg (43.39 KB, \u4e0b\u8f7d\u6b21\u6570: 0)  \u4e0b\u8f7d\u9644\u4ef6  \u4fdd\u5b58\u5230\u76f8\u518c  19-10-2015 01:13 PM \u4e0a\u4f20", {"english": "Xuesha's face is so long...", "malay": "Muka Xuesha sangat panjang..."}]
["haha~~  \u6211\u6765\u4e5f\uff01  \u53ea\u53ef\u60dc\u6211\u6ca1\u6709\u5174\u8da3~~", {"english": "Haha~~ I'm here too! Unfortunately, I'm not interested~~", "malay": "Haha~~ Saya juga ada di sini! Malangnya, saya tidak berminat~~"}]


In [4]:
import json

data = []
with open('c.cari.com.my.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [6]:
len(data)

744371

In [7]:
import re

def simple(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

def reject(k):
    if 'saya tidak dapat menterjemah teks' in k.lower():
        return
    if 'saya tidak boleh menterjemah kandungan yang tidak sesuai' in k.lower():
        return
    if 'saya tidak boleh menterjemah teks' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai makna' in k.lower():
        return
    if 'teks yang disediakan tidak boleh diterjemahkan' in k.lower():
        return
    if 'teks yang diberikan tidak masuk akal' in k.lower():
        return
    if 'teks yang diberikan nampaknya tidak' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai maksud' in k.lower():
        return
    if 'teks yang diberikan bukan dalam mana-mana bahasa' in k.lower():
        return
    if 'teks ini tidak boleh diterjemahkan' in k.lower():
        return
    if 'saya tidak dapat menterjemah frasa' in k.lower():
        return
    if 'teks yang disediakan bukan' in k.lower():
        return
    if 'teks yang diberikan tidak jelas' in k.lower():
        return
    if 'teks yang diberikan tidak' in k.lower():
        return
    if 'terjemahan teks kepada' in k.lower():
        return
    if 'saya tidak boleh menterjemah bahasa' in k.lower():
        return
    if 'model bahasa AI' in k:
        return
    if 'bahasa melayu standard' in k.lower():
        return
    if 'return JSON structure' in k:
        return
    if 'teks yang diberikan bukan dalam bahasa' in k.lower():
        return
    if 'teks yang anda berikan bukan' in k.lower():
        return
    if 'teks yang disediakan tidak' in k.lower():
        return
    if 'teks yang diberikan bukan' in k.lower():
        return
    if 'saya tidak dapat menterjemah' in k.lower():
        return
    if 'tetapi teks yang diberikan' in k.lower():
        return
    
    return True

In [15]:
from tqdm import tqdm

filtered = []
for d in tqdm(data):
    if d is None:
        continue
    if 'src' in d:
        left = d['src']['text']
        try:
            en = d['r']['english']
            ms = d['r']['malay']
        except:
            en = None
            ms = None
    else:
        left = d[0]
        success = False
        try:
            r = json.loads(d[1])
            success = True
        except:
            pass
        
        if not success:
            try:
                r = eval(d[1])
                success = True
            except:
                pass
            
        if not success and type(d[1]) == dict:
            r = d[1]
            success = True
        
        en = None
        ms = None
        
        try:
            if success:
                if isinstance(r, tuple):
                    en = [r_['english'] for r_ in r]
                    ms = [r_['malay'] for r_ in r]
                    en = ' '.join(en)
                    ms = ' '.join(ms)
                else:
                    en = r['english']
                    ms = r['malay']
        except:
            pass
        
    if en and not type(en) == str:
        continue
        
    simple_left = simple(left)
    
    if (en and simple(en) == simple_left) or (ms and simple(ms) == simple_left):
        continue
    
    if en or ms:
        filtered.append({
            'left': left,
            'en': en,
            'ms': ms
        })           

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 744371/744371 [00:18<00:00, 39552.47it/s]


In [16]:
data[1]

[' xuesha的脸好长啊。。。  10507975_338882742936189_1743878600_n.jpg (43.39 KB, 下载次数: 0)  下载附件  保存到相册  19-10-2015 01:13 PM 上传',
 {'english': "Xuesha's face is so long...",
  'malay': 'Muka Xuesha sangat panjang...'}]

In [17]:
len(filtered)

740826

In [18]:
len(filtered), len(data)

(740826, 744371)

In [19]:
with open('processed-c.cari.com.my.jsonl', 'w') as fopen:
    for d in filtered:
        try:
            if not reject(d['ms']):
                continue
            fopen.write(f'{json.dumps(d)}\n')
        except:
            pass

In [20]:
!wc -l processed-c.cari.com.my.jsonl

740431 processed-c.cari.com.my.jsonl


In [22]:
!head -n 10 processed-c.cari.com.my.jsonl

{"left": "\u56de\u590d 1# jeffreylee  \u53ef\u4ee5\u7528\u4e00\u9910 value meal \u4f86\u63db\u55ce\uff1f", "en": "Can I exchange it with a value meal?", "ms": "Bolehkah saya menukar ini dengan hidangan nilai?"}
{"left": " xuesha\u7684\u8138\u597d\u957f\u554a\u3002\u3002\u3002  10507975_338882742936189_1743878600_n.jpg (43.39 KB, \u4e0b\u8f7d\u6b21\u6570: 0)  \u4e0b\u8f7d\u9644\u4ef6  \u4fdd\u5b58\u5230\u76f8\u518c  19-10-2015 01:13 PM \u4e0a\u4f20", "en": "Xuesha's face is so long...", "ms": "Muka Xuesha sangat panjang..."}
{"left": "haha~~  \u6211\u6765\u4e5f\uff01  \u53ea\u53ef\u60dc\u6211\u6ca1\u6709\u5174\u8da3~~", "en": "Haha~~ I'm here too! Unfortunately, I'm not interested~~", "ms": "Haha~~ Saya juga ada di sini! Malangnya, saya tidak berminat~~"}
{"left": "\u771f\u7684\u5f88\u611f\u6fc0\u4f60\u3002\u4f60\u4e5f\u662f\u4f4fCheras\u5417\uff1f\u8981\u4e00\u8d77\u53bb\u5417\uff1f  \u6211\u6709\u6536\u5230\u4f60\u7684\u597d\u53cb\u9080\u8bf7\uff0c\u53ef\u662f\u6211\u4e0d\u4f1aaccept\